# Creating a Siamese Network

Creating a Siamese Network Model in Keras with a custom layer and a custom loss function involves several steps. We'll start by defining the custom layer and the custom loss function, and then proceed to build the Siamese Network.

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Layer, Input, Dense, Flatten, Lambda
from tensorflow.keras.models import Model

class CustomLayer(Layer):
    def __init__(self, units=32, **kwargs):
        super(CustomLayer, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w)


In [ ]:
def contrastive_loss(y_true, y_pred):
    margin = 1
    y_true = tf.cast(y_true, dtype=tf.float32)
    square_pred = tf.square(y_pred)
    margin_square = tf.square(tf.maximum(margin - y_pred, 0))
    return tf.reduce_mean(y_true * square_pred + (1 - y_true) * margin_square)


A Siamese Network consists of two identical subnetworks with shared weights.

In [ ]:
def build_base_network(input_shape):
    input = Input(shape=input_shape)
    x = Flatten()(input)
    x = Dense(128, activation='relu')(x)
    x = CustomLayer(64)(x)
    return Model(input, x)

# Assuming MNIST images of shape 28x28
input_shape = (28, 28)

# Create the base network
base_network = build_base_network(input_shape)

# Create the left input and point to the base network
input_a = Input(shape=input_shape)
processed_a = base_network(input_a)

# Create the right input and point to the base network
input_b = Input(shape=input_shape)
processed_b = base_network(input_b)

# Add a distance metric, e.g., Euclidean distance
distance = Lambda(lambda embeddings: tf.sqrt(tf.reduce_sum(tf.square(embeddings[0] - embeddings[1]), axis=1, keepdims=True)))(
    [processed_a, processed_b])

# Create the Siamese Network model
model = Model([input_a, input_b], distance)

# Compile the model
model.compile(loss=contrastive_loss, optimizer='adam')


Let's create some dummy data to train the Siamese Network. In a real-world scenario, this would be pairs of images and a label indicating whether they are similar or not. For simplicity, we will create random numpy arrays as dummy data. The concept remains the same for actual image data.

In [ ]:
# Generate dummy data
def generate_dummy_data(num_samples=1000, input_shape=(28, 28)):
    data = np.random.random((num_samples, *input_shape))
    labels = np.random.randint(0, 2, (num_samples, 1))
    return data, labels

num_samples = 1000
input_shape = (28, 28)

# Generate pairs of dummy data
data_a, labels_a = generate_dummy_data(num_samples, input_shape)
data_b, labels_b = generate_dummy_data(num_samples, input_shape)

# Generating labels (1 if pairs are similar, 0 if they are different)
labels = np.equal(labels_a, labels_b).astype(float)

# Train the model
model.fit([data_a, data_b], labels, batch_size=128, epochs=10)

Epoch 1/10
8/8 [==============================] - 1s 7ms/step - loss: 0.5261
Epoch 2/10
8/8 [==============================] - 0s 6ms/step - loss: 0.2496
Epoch 3/10
8/8 [==============================] - 0s 6ms/step - loss: 0.2257
Epoch 4/10
8/8 [==============================] - 0s 7ms/step - loss: 0.2021
Epoch 5/10
8/8 [==============================] - 0s 7ms/step - loss: 0.1787
Epoch 6/10
8/8 [==============================] - 0s 10ms/step - loss: 0.1572
Epoch 7/10
8/8 [==============================] - 0s 6ms/step - loss: 0.1383
Epoch 8/10
8/8 [==============================] - 0s 7ms/step - loss: 0.1212
Epoch 9/10
8/8 [==============================] - 0s 7ms/step - loss: 0.1068
Epoch 10/10
8/8 [==============================] - 0s 6ms/step - loss: 0.0941
